<a href="https://colab.research.google.com/github/VR952004/Interlagos-Racing-Line/blob/main/Interlagos_Racing_Line.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import os
from sklearn.pipeline import Pipeline
from tensorflow.keras import layers, models
from sklearn.preprocessing import FunctionTransformer

In [2]:
train_data_path = '/content/drive/My Drive/Interlagos Racing Line/training_data/'
test_data_path = '/content/drive/My Drive/Interlagos Racing Line/test_data/'

In [3]:
def extract_frames_from_video(video_path, frame_interval=2):
    """
    Extract frames from the video at a specified interval.

    :param video_path: Path to the input video.
    :param frame_interval: Interval at which to sample frames (e.g., every nth frame).
    :return: A numpy array of extracted frames.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []

    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return None

    # Get the video's FPS and total number of frames
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Extract only every nth frame, based on the frame_interval
        if frame_count % frame_interval == 0:
            frames.append(frame)

        frame_count += 1

    cap.release()
    frames = np.array(frames)
    return frames

In [4]:
# Step 2: Preprocessing step: resize and normalize the frames
def preprocess_frames(frames, frame_size=(224, 224)):
    processed_frames = []
    for frame in frames:
        # Resize the frame
        resized_frame = cv2.resize(frame, frame_size)

        # Convert to grayscale
        gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

        # Normalize the frame
        normalized_frame = gray_frame / 255.0

        # Reshape and append the frame
        processed_frames.append(normalized_frame.reshape(*frame_size, 1))

    return np.array(processed_frames)

In [5]:
# Using FunctionTransformer to turn the preprocessing into a pipeline step
extract_frames_transformer = FunctionTransformer(extract_frames_from_video)
preprocess_transformer = FunctionTransformer(preprocess_frames, kw_args={"frame_size": (224, 224)})

In [6]:
# Step 3: Define the Autoencoder model structure
def build_autoencoder(input_shape=(224, 224, 1)):
    input_img = layers.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    latent_space = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(latent_space)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)

    # Output layer
    decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    # Compile the model
    autoencoder = models.Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')

    return autoencoder

In [7]:
# Build the autoencoder model
autoencoder = build_autoencoder()

In [8]:
# Custom wrapper for the autoencoder
class AutoencoderWrapper:
    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        # Fit the autoencoder on the data
        self.model.fit(X, X, epochs=10, batch_size=256, validation_split=0.2)
        return self

    def transform(self, X):
        # Use the autoencoder to reconstruct the frames
        return self.model.predict(X)

# Build the full pipeline: extract frames -> preprocess frames -> autoencoder
pipeline = Pipeline(steps=[
    ('extract_frames', extract_frames_transformer),
    ('preprocess_frames', preprocess_transformer),
    ('autoencoder', AutoencoderWrapper(autoencoder))
])

In [9]:
# Step 4: Process all videos in the training_data folder
all_frames = []

# Iterate over all video files in the training_data folder
for video_file in os.listdir(train_data_path):
    video_path = os.path.join(train_data_path, video_file)

    # Run the pipeline to extract, preprocess, and store frames
    print(f"Processing {video_file}")
    frames = pipeline.named_steps['extract_frames'].transform(video_path)
    processed_frames = pipeline.named_steps['preprocess_frames'].transform(frames)

    # If frames were processed successfully, add them to all_frames
    if processed_frames is not None:
        all_frames.append(processed_frames)

# Combine all frames from all videos into one NumPy array
train_frames = np.concatenate(all_frames, axis=0)
print(f"Total training frames: {train_frames.shape[0]}")

# Step 5: Train the autoencoder using the processed frames
pipeline.named_steps['autoencoder'].fit(train_frames)

Processing processed_onboard1.mp4
Processing processed_onboard2.mp4
Processing processed_onboard3.mp4
Processing processed_onboard4.mp4
Processing processed_onboard5.mp4
Processing processed_onboard6.mp4
Processing processed_onboard7.mp4
Processing processed_onboard8.mp4
Processing processed_onboard9.mp4
Total training frames: 26019
Epoch 1/10
82/82 [==============================] - 846s 10s/step - loss: 0.0105 - val_loss: 0.0027
Epoch 2/10
82/82 [==============================] - 837s 10s/step - loss: 0.0028 - val_loss: 0.0017
Epoch 3/10
82/82 [==============================] - 841s 10s/step - loss: 0.0021 - val_loss: 0.0013
Epoch 4/10
82/82 [==============================] - 843s 10s/step - loss: 0.0018 - val_loss: 0.0013
Epoch 5/10
82/82 [==============================] - 847s 10s/step - loss: 0.0015 - val_loss: 0.0010
Epoch 6/10
82/82 [==============================] - 847s 10s/step - loss: 0.0014 - val_loss: 9.6508e-04
Epoch 7/10
82/82 [==============================] - 849s 10s/

In [10]:
import joblib

# Save the entire pipeline (preprocessing + autoencoder)
pipeline_file = '/content/drive/My Drive/Interlagos Racing Line/racingline_complete_pipeline.pkl'
joblib.dump(pipeline, pipeline_file)

print(f"Complete pipeline saved to {pipeline_file}")

Complete pipeline saved to /content/drive/My Drive/Interlagos Racing Line/racingline_complete_pipeline.pkl
